# Save tiff stacks from individual hdf5 files

In [7]:
import h5py
import os
import glob
import numpy as np
import tifffile

In [8]:
folder_top = 'example_data'
dirs_top = []
for item in os.listdir(folder_top):
    if os.path.isdir(os.path.join(folder_top, item)) and 'nontriggered' not in item and 'false' not in item:
        dirs_top.append(os.path.join(folder_top, item))
print(dirs_top)

savetifs = True
savelogs = True

for folder in dirs_top:

    pipelinetime = []

    if 'stedresolution' in folder or 'triggeredtimelapses' in folder or 'stedtimelapse' in folder:
        dirs = [x[0] for x in os.walk(folder)][1:]

        print(len(dirs))

        for recdir in dirs:
            if 'false' not in recdir and '\false' not in recdir and 'rest' not in recdir:
                wffiles = []
                stedfiles = []
                logfiles = []

                for file in os.listdir(recdir):
                    if file.endswith('Camera.hdf5'):
                        wffiles.append(os.path.join(recdir, file))
                    elif file.endswith('APDgreen.hdf5'):
                        stedfiles.append(os.path.join(recdir, file))
                    elif file.endswith('log.txt'):
                        logfiles.append(os.path.join(recdir, file))

                #print(recdir)
                #print(wffiles)
                #print(stedfiles)

                if savetifs:
                    f0 = h5py.File(wffiles[0], 'r')
                    dset0 = f0["data:smartSTEDCamera"][...]
                    datasetshape = np.shape(dset0)
                    stackwf = np.empty((len(wffiles),datasetshape[0],datasetshape[1]))
                    wfsavename = os.path.join(recdir.rsplit('\\',1)[0], recdir.rsplit('\\',1)[1]+'-smartSTEDCamera-raw.tif')

                    f0 = h5py.File(stedfiles[0], 'r')
                    dset0 = f0["data:APDgreen"][...]
                    datasetshape = np.shape(dset0)
                    stacksted = np.empty((len(stedfiles),datasetshape[0],datasetshape[1]))
                    stedsavename = os.path.join(recdir.rsplit('\\',1)[0], recdir.rsplit('\\',1)[1]+'-APDgreen-STED.tif')

                    for idx, file in enumerate(wffiles):
                        f = h5py.File(file, 'r')
                        dset = f["data:smartSTEDCamera"][...]
                        stackwf[idx] = dset
                    for idx, file in enumerate(stedfiles):
                        f = h5py.File(file, 'r')
                        dset = f["data:APDgreen"][...]
                        stacksted[idx] = dset
                    tifffile.imwrite(wfsavename, stackwf)
                    tifffile.imwrite(stedsavename, stacksted)

                if savelogs:
                    f0 = open(logfiles[0], 'r')
                    lines = f0.readlines()
                    pipelinetimes = [0,0]
                    for line in lines:
                        param = line.split(' ')[0]
                        if param == 'pipeline_start:':
                            pipelinetimes[0] = float(line.split(' ')[1].split('\n')[0].split('s')[0]) + float(line.split(' ')[1].split('\n')[0].split('s')[1].split('u')[0])/1e6
                        elif param == 'pipeline_end:':
                            pipelinetimes[1] = float(line.split(' ')[1].split('\n')[0].split('s')[0]) + float(line.split(' ')[1].split('\n')[0].split('s')[1].split('u')[0])/1e6
                    pipelinetime.append((pipelinetimes[1]-pipelinetimes[0])*1000)  # in ms

    elif 'largestedtimelapses' in folder:
        dirs = [x[0] for x in os.walk(folder)][1:]

        print(len(dirs))

        for recdir in dirs:
            if 'false' not in recdir and '\false' not in recdir and 'rest' not in recdir:
                wffiles = []
                stedfiles = []
                logfiles = []

                for file in os.listdir(recdir):
                    if file.endswith('APDgreen.hdf5'):
                        stedfiles.append(os.path.join(recdir, file))
                    elif file.endswith('log.txt'):
                        logfiles.append(os.path.join(recdir, file))

                if savetifs:
                    f0 = h5py.File(stedfiles[0], 'r')
                    dset0 = f0["data"][...]
                    datasetshape = np.shape(dset0)
                    stacksted = np.empty((len(stedfiles),datasetshape[1],datasetshape[2]))
                    stedsavename = os.path.join(recdir.rsplit('\\',1)[0], recdir.rsplit('\\',1)[1]+'-APDgreen-STED.tif')

                    for idx, file in enumerate(stedfiles):
                        f = h5py.File(file, 'r')
                        dset = f["data"][...]
                        stacksted[idx] = dset

                    tifffile.imwrite(stedsavename, stacksted)

                if savelogs:
                    f0 = open(logfiles[0], 'r')
                    lines = f0.readlines()
                    pipelinetimes = [0,0]
                    for line in lines:
                        param = line.split(' ')[0]
                        if param == 'pipeline_start:':
                            pipelinetimes[0] = float(line.split(' ')[1].split('\n')[0].split('s')[0]) + float(line.split(' ')[1].split('\n')[0].split('s')[1].split('u')[0])/1e6
                        elif param == 'pipeline_end:':
                            pipelinetimes[1] = float(line.split(' ')[1].split('\n')[0].split('s')[0]) + float(line.split(' ')[1].split('\n')[0].split('s')[1].split('u')[0])/1e6
                    pipelinetime.append((pipelinetimes[1]-pipelinetimes[0])*1000)  # in ms

    elif 'validationruns' in folder:
        dirs = [x[0] for x in os.walk(folder)][1:]

        print(len(dirs))

        for recdir in dirs:
            if 'false' not in recdir and '\false' not in recdir and 'rest' not in recdir:
                rawfiles = []
                anafiles = []
                logfiles = []

                for file in os.listdir(recdir):
                    if file.endswith('Camera.hdf5'):
                        if 'raw' in file:
                            rawfiles.append(os.path.join(recdir, file))
                        elif 'ana' in file:
                            anafiles.append(os.path.join(recdir, file))
                    elif file.endswith('log.txt'):
                        logfiles.append(os.path.join(recdir, file))

                if savetifs:
                    f0 = h5py.File(rawfiles[0], 'r')
                    dset0 = f0["data:smartSTEDCamera"][...]
                    datasetshape = np.shape(dset0)
                    stackraw = np.empty((len(rawfiles),datasetshape[0],datasetshape[1]))
                    rawsavename = os.path.join(recdir.rsplit('\\',1)[0], recdir.rsplit('\\',1)[1]+'-raw.tif')

                    f0 = h5py.File(anafiles[0], 'r')
                    dset0 = f0["data:smartSTEDCamera"][...]
                    datasetshape = np.shape(dset0)
                    stackana = np.empty((len(anafiles),datasetshape[0],datasetshape[1]))
                    anasavename = os.path.join(recdir.rsplit('\\',1)[0], recdir.rsplit('\\',1)[1]+'-ana.tif')

                    for idx, file in enumerate(rawfiles):
                        f = h5py.File(file, 'r')
                        dset = f["data:smartSTEDCamera"][...]
                        stackraw[idx] = dset
                    for idx, file in enumerate(anafiles):
                        f = h5py.File(file, 'r')
                        dset = f["data:smartSTEDCamera"][...]
                        stackana[idx] = dset
                    tifffile.imwrite(rawsavename, stackraw)
                    tifffile.imwrite(anasavename, stackana)

                if savelogs:
                    f0 = open(logfiles[0], 'r')
                    lines = f0.readlines()
                    pipelinetimes = [0,0]
                    for line in lines:
                        param = line.split(' ')[0]
                        if param == 'pipeline_start:':
                            pipelinetimes[0] = float(line.split(' ')[1].split('\n')[0].split('s')[0]) + float(line.split(' ')[1].split('\n')[0].split('s')[1].split('u')[0])/1e6
                        elif param == 'pipeline_end:':
                            pipelinetimes[1] = float(line.split(' ')[1].split('\n')[0].split('s')[0]) + float(line.split(' ')[1].split('\n')[0].split('s')[1].split('u')[0])/1e6
                        if param == 'fastscan_x_center:' or param == 'fastscan_y_center:':
                            print(line.split(' ')[1])
                    pipelinetime.append((pipelinetimes[1]-pipelinetimes[0])*1000)  # in ms
                print('')

    else:
        print('Only works for stedresolution and triggeredtimelapses arranged files right now.')

    if savelogs:
        print(pipelinetime)
        pipelinetimesavename = os.path.join(folder, 'pipelinetimes.txt')
        with open(pipelinetimesavename, 'w') as f:
            for item in pipelinetime:
                f.write("%s\n" % item)

['example_data\\area001_triggeredtimelapses']
2
[6.001000000001255, 8.001999999997622]
